In [355]:
import pandas as pd
import numpy as np 
import duckdb as dd
import re

In [356]:
# Create an in-memory DuckDB connection
conn = dd.connect(':memory:')

In [357]:
query = """
SELECT
       *,                                                -- every column in the CSV
       CAST(
            regexp_extract(filename,
                           '/sept-yearling/([0-9]{4})/', -- capture the 4‑digit dir
                           1)                             -- first capture group
            AS INTEGER)           AS sale_year
FROM read_csv_auto(
       '../data/keeneland/sept-yearling/*/lots.csv',
       FILENAME = TRUE        -- adds a “filename” column with the full path
);
"""
df = conn.sql(query).df()

In [358]:
df.head(5)

,Session,Hip,PropertyLine1,PropertyLine2,Description,DOB,Color,Sex,Horse Name,Sire,Dam,CoveringSire,LastService,Pregnancy,SoldAs,Purchaser,Price,filename,sale_year
0,1,0001,"Lane's End, Agent",None,Concord Bridge,None,B,Colt,Concord Bridge,Medaglia d'Oro,Pauline Revere,None,None,None,None,Talla Racing & West Point Thoroughbreds,850000,../data/keeneland/sept-yearling/2022/lots.csv,2022
1,1,0002,Summerfield (Francis and Barbara Vanlangendonc...,None,Uncoachable,None,B,Colt,Uncoachable,Into Mischief,Peggy Jane,None,None,None,None,Out,0,../data/keeneland/sept-yearling/2022/lots.csv,2022
2,1,0003,"Gainesway, Agent X",None,Reserve Fund,None,CH,Filly,Reserve Fund,Tapit,Pension,None,None,None,None,"Steven W. Young, Agent",650000,../data/keeneland/sept-yearling/2022/lots.csv,2022
3,1,0004,Dixiana Farms LLC,None,Paparazzo,None,B,Colt,Paparazzo,American Pharoah,Photographer,None,None,None,None,"Quarter Pole Enterprises, LLC",200000,../data/keeneland/sept-yearling/2022/lots.csv,2022
4,1,0005,"Taylor Made Sales Agency, Agent LXI",None,Blood Brother,None,B,Colt,Blood Brother,Quality Road,Pine Needles,None,None,None,None,Calumet Farm,100000,../data/keeneland/sept-yearling/2022/lots.csv,2022


## Sellers Preprocessing
- Hip and sale_year uniquely identify horse
- seller_detail: full seller with agent info stripped
    - This may include a farm "Summerfield" along with the farm owners names "(Francis and Barbara Vanlangendonck)"
- primary_seller: using the example above, this would just be the farm "Summerfield"
- seller_affiliation: this would be Francis and Barbara Vanlangendonck
    - It's possible the affiliation and the primary seller are the same thing, but because farms and ownership can change hands, we will anchor on whatever comes first. 
    - Note: there may be errors here because names sometimes preceed farms. In that case, we would assume the names indicate owners who have multiple farms. 
- agent: the agent associated with the transaction. 
    - No agent listed when the owner is also the seller
- owner_detail: full ownership information typically occuring if we see "Agent for..." or no "Agent" at all
    - This can include and individual and farm or vice versa "Ann Mudge Backer (Smitten Farm)"
- primary_owner: using the example above, "Ann Mudge Backer"
- owner_affiliation: using example above, "Smitten Farm"

In [359]:
# Ex Seller: 'Summerfield (Francis and Barbara Vanlangendonck), Agent for Stonestreet Bred & Raised'
data = {"Hip": [],
        "sale_year": [],
        "seller_detail": [], # equals Summerfield (Francis and Barbara Vanlangendonck), always populated
        "primary_seller": [], # equals Summerfield; always populated
        "seller_affiliation": [], # equals (Francis and Barbara Vanlangendonck); null is nothing in parentheses
        "agent": [], # the listed agent which will always be paired with primary seller
        "owner_detail": [],
        "primary_owner": [],
        "owner_affiliation": []
       } 

# Parse
for idx, row in df.iterrows():
    # Append hip, sale_year, seller_detail which will be the same regardless of criteria
    data["Hip"].append(row["Hip"])
    data["sale_year"].append(row["sale_year"])
    
    # check for ", Agent"
    # If Agent does not appear then we know OWNER and can fill in all data
    if ", Agent" not in row["PropertyLine1"]:
        # Seller detail and owner detail are the same
        data["seller_detail"].append(row["PropertyLine1"].strip())
        data["owner_detail"].append(row["PropertyLine1"].strip())
        data["agent"].append(None)

        # Check for parentheses
        if re.findall("\((.*?)\)", row["PropertyLine1"]):
            # Fill affiliation
            affiliation = re.findall("\((.*?)\)", row["PropertyLine1"])[0]
            data["seller_affiliation"].append(affiliation)
            data["owner_affiliation"].append(affiliation)
            
            # Fill primary seller name
            p_affiliation = "(" + affiliation + ")"
            data["primary_seller"].append(row["PropertyLine1"].replace(p_affiliation, "").strip())
            data["primary_owner"].append(row["PropertyLine1"].replace(p_affiliation, "").strip())
        else:
            data["seller_affiliation"].append(None)
            data["primary_seller"].append(row["PropertyLine1"].strip())
            data["owner_affiliation"].append(None)
            data["primary_owner"].append(row["PropertyLine1"].strip())
   
    # ", Agent" does appear - means we know seller info, we MAY know OWNER
    else: 
        # Get seller info
        # This split logic either breaks the seller/owner info into "..., Agent X" or "..." with "Agent for ..." leftover
        seller_info = row["PropertyLine1"].split(', Agent for')[0].strip()
        data["seller_detail"].append(seller_info)
        
        # Process known OWNER
        # Is there evidence of an owner given by "Agent for"
        if ", Agent for" in row["PropertyLine1"]:
            # Split owner data (already have seller data)
            owner_info = row["PropertyLine1"].split(', Agent for')[1].strip()
            data["owner_detail"].append(owner_info)
            
            # Append generic agent
            data["agent"].append("Agent")
            
            # OWNER 
            # check for parentheses - multi part owner? Calumet Farm (Amy Reed)
            if re.findall("\((.*?)\)", owner_info):
                p_affiliation = "(" + re.findall("\((.*?)\)", owner_info)[0] + ")"
                data["primary_owner"].append(owner_info.replace(p_affiliation, "").strip())
                data["owner_affiliation"].append(re.findall("\((.*?)\)", owner_info)[0])
            else: 
                data["primary_owner"].append(owner_info)
                data["owner_affiliation"].append(None)
                
            # SELLER
            if re.findall("\((.*?)\)", seller_info):
                s_affiliation = "(" + re.findall("\((.*?)\)", seller_info)[0] + ")"
                data["primary_seller"].append(seller_info.replace(s_affiliation, "").strip())
                data["seller_affiliation"].append(re.findall("\((.*?)\)", seller_info)[0])
            else: 
                data["primary_seller"].append(seller_info)
                data["seller_affiliation"].append(None)
            
        else:
            # If "Agent for" is missing then we won't be able to identify the owner
            data["owner_detail"].append(None)
            data["primary_owner"].append(None)
            data["owner_affiliation"].append(None)
        
            # Process SELLER for unknown owner
            # Check for agent
            if ', Agent' in seller_info:
                agent = 'Agent ' + seller_info.split(', Agent')[1].strip()
                seller = seller_info.split(', Agent')[0].strip()
                data["agent"].append(agent.strip())

                # Check for parentheses [i.e. Calumet Farm (Amy Reed), Agent X]
                if re.findall("\((.*?)\)", seller):
                    # Fill affiliation
                    affiliation = re.findall("\((.*?)\)", seller)[0]
                    data["seller_affiliation"].append(affiliation)
                    data["primary_seller"].append(seller.replace("(" + affiliation + ")", "").strip())
                else:
                    data["seller_affiliation"].append(None)
                    data["primary_seller"].append(seller.strip())
            else:
                pass

In [360]:
for k, v in data.items():
    print(k, len(v))

Hip 30249
sale_year 30249
seller_detail 30249
primary_seller 30249
seller_affiliation 30249
agent 30249
owner_detail 30249
primary_owner 30249
owner_affiliation 30249


In [361]:
sellers = pd.DataFrame(data)
sellers.to_csv('sellers.csv', index=False)

## Buyers
This requires a preliminary cleaning of the Purchaser column. The column can take on the following formats: 
- Laura McKinney
- R.N.A. (325,000)
- Out
- R.N.A. (0)
    - Presumably went to the ring but was unsold
- Kenneth McPeek, Agent
    - If Agent appears, then it means the entire line is an agent purchase. So Dale Romans, Kenneth McPeek, Agent would be split into "Dale Romans, Kenneth McPeek"
- Kenneth McPeek, Agent for Godolphin
    - In this case, we actually know the owner and the agent

I need to capture the following bits of post-processed information:
1. Hip - known
2. sale_year - known
3. sales_status - simple logic
    - rna if R.N.A. and (\*) != 0
    - out if Out
    - unsold if R.N.A (0)
    - sold if anything else
4. sale_price - dependent logic on 3
    - Price if sales_status == 'sold' else np.nan()
5. known_rna_price
    - \* in R.N.A. (\*) if \* != 0 else np.nan()
6. buyer_owner
    - if 'Agt for' or 'Agent for' in Purchaser then buyer_owner = purchaser.split('Agt for')[1].strip() | purchaser.split('Agent for')[1].strip() else None
    - if 'Agt' and 'Agent' not in Purchaser then purchaser else None
7. buyer_agent
    - if len(purchaser.split(', Agent')[1])==0 then purchaser.split(', Agent')[0] else None

In [362]:
data = {"Hip": [],
        "sale_year": [],
        "sales_status": [], # equals Summerfield (Francis and Barbara Vanlangendonck), always populated
        "sale_price": [], # equals Summerfield; always populated
        "known_rna_price": [], # equals (Francis and Barbara Vanlangendonck); null is nothing in parentheses
        "buyer_agent": [],
        "buyer_owner_detail": [],
        "primary_buyer_owner": [],
        "buyer_owner_affiliation": [] # the listed agent which will always be paired with primary seller
       } 

for idx, row in df.iterrows():
    data["Hip"].append(row["Hip"])
    data["sale_year"].append(row["sale_year"])
    
    # Sales price
    data["sale_price"].append(row["Price"] if is_int(row["Price"]) and row["Price"]!="0" else None)
    
    # Sale status and known_rna_price
    if row["Price"]=="0":
        data["sales_status"].append("out")
        data["known_rna_price"].append(None)
    elif is_int(row["Price"]):
        data["sales_status"].append("sold")
        data["known_rna_price"].append(None)
    elif row["Purchaser"].split(" (")[1].strip(")")=="0":
        data["sales_status"].append("unsold")
        data["known_rna_price"].append(None)
    else:
        data["sales_status"].append("rna")
        data["known_rna_price"].append(row["Purchaser"].split("(")[1].strip(")").replace(",", ""))
        
    # Buyer Agent and Owner
    if "R.N.A." in row["Purchaser"] or row["Purchaser"]=='Out':
        data["buyer_agent"].append(None)
        data["buyer_owner_detail"].append(None)
        data["primary_buyer_owner"].append(None)
        data["buyer_owner_affiliation"].append(None)

    elif "Agent" in row["Purchaser"]:
        data["buyer_agent"].append(row["Purchaser"].split("Agent")[0].strip(", "))
        if "Agent for" in row["Purchaser"]:
            owner_detail = row["Purchaser"].split("Agent for ")[1].strip()
            data["buyer_owner_detail"].append(owner_detail)
            if re.findall("\((.*?)\)", owner_detail):
                affiliation = re.findall("\((.*?)\)", owner_detail)[0]
                data["buyer_owner_affiliation"].append(affiliation)
                data["primary_buyer_owner"].append(owner_detail.replace("(" + affiliation + ")", "").strip())
            else:
                data["buyer_owner_affiliation"].append(None)
                data["primary_buyer_owner"].append(owner_detail)
        else: 
            data["buyer_owner_detail"].append(None)
            data["buyer_owner_affiliation"].append(None)
            data["primary_buyer_owner"].append(None)
            
    elif "Agt" in row["Purchaser"]:
        data["buyer_agent"].append(row["Purchaser"].split("Agt")[0].strip(", "))
        if "Agt for" in row["Purchaser"]:
            owner_detail = row["Purchaser"].split("Agt for ")[1].strip()
            data["buyer_owner_detail"].append(owner_detail)
            if re.findall("\((.*?)\)", owner_detail):
                affiliation = re.findall("\((.*?)\)", owner_detail)[0]
                data["buyer_owner_affiliation"].append(affiliation)
                data["primary_buyer_owner"].append(owner_detail.replace("(" + affiliation + ")", "").strip())
            else:
                data["buyer_owner_affiliation"].append(None)
                data["primary_buyer_owner"].append(owner_detail)
        else:
            data["buyer_owner_detail"].append(None)
            data["buyer_owner_affiliation"].append(None)
            data["primary_buyer_owner"].append(None)
    
    else: # Assume all we have is owner data; need to check for affiliation
        detail = row["Purchaser"]
        data["buyer_agent"].append(None)
        data["buyer_owner_detail"].append(detail)
        
        if re.findall("\((.*?)\)", detail):
            affiliation = re.findall("\((.*?)\)", detail)[0]
            data["buyer_owner_affiliation"].append(affiliation)
            data["primary_buyer_owner"].append(detail.replace("(" + affiliation + ")", "").strip())
        else:
            data["buyer_owner_affiliation"].append(None)
            data["primary_buyer_owner"].append(detail)

In [363]:
for k,v in data.items():
    print(k, len(v))

Hip 30249
sale_year 30249
sales_status 30249
sale_price 30249
known_rna_price 30249
buyer_agent 30249
buyer_owner_detail 30249
primary_buyer_owner 30249
buyer_owner_affiliation 30249


In [364]:
buyers = pd.DataFrame(data)
buyers.to_csv('buyers.csv', index=False)

## Join to master

In [365]:
final = df.merge(sellers, how="left", on=["Hip", "sale_year"]).merge(buyers, how="left", on=["Hip", "sale_year"])

In [366]:
final.columns

Index(['Session', 'Hip', 'PropertyLine1', 'PropertyLine2', 'Description',
       'DOB', 'Color', 'Sex', 'Horse Name', 'Sire', 'Dam', 'CoveringSire',
       'LastService', 'Pregnancy', 'SoldAs', 'Purchaser', 'Price', 'filename',
       'sale_year', 'seller_detail', 'primary_seller', 'seller_affiliation',
       'agent', 'owner_detail', 'primary_owner', 'owner_affiliation',
       'sales_status', 'sale_price', 'known_rna_price', 'buyer_agent',
       'buyer_owner_detail', 'primary_buyer_owner', 'buyer_owner_affiliation'],
      dtype='object')

In [367]:
cols_to_keep = ['Session', 'Hip', 'Description', 'DOB', 'Color', 'Sex', 
                'Horse Name', 'Sire', 'Dam', 'sale_year', 
                'seller_detail', 'primary_seller', 'seller_affiliation',
                'agent', 'owner_detail', 'primary_owner', 'owner_affiliation',
                'sales_status', 'sale_price', 'known_rna_price', 'buyer_agent',
                'buyer_owner_detail', 'primary_buyer_owner', 'buyer_owner_affiliation']
final = final[cols_to_keep]

In [368]:
final.to_csv('yearlings_buyers_sellers.csv', index=False)

In [353]:
df.loc[583, :]

Session                                                          3
Hip                                                           0595
PropertyLine1                             Clearsky Farms, Agent I 
PropertyLine2                                                 None
Description                                     Determined Justine
DOB                                                           None
Color                                                            B
Sex                                                          Filly
Horse Name                                      Determined Justine
Sire                                                       Justify
Dam                                                         Coniah
CoveringSire                                                  None
LastService                                                   None
Pregnancy                                                     None
SoldAs                                                        